# Etude Snorkel / Extraction de partenariats d'entreprises
Pour ce projet, nous allons essayer d'extraire des relations de partenariats économiques entre des entreprises depuis des articles. <br/>
Le but étant de comprendre l'utilisation de snorkel.

Nous allons découper notre traitement dans 3 notebooks (chacun correspondant à une étape du pipeline `Snorkel` :
1. Preprocessing
2. Training
3. Evaluation

## Partie III: Entrainement d'un modèle d'extraction final

Dans ce notebook, nous allons utiliser nos labels (bruités) que nous avons générés dans le notebook précédent pour entrainer notre modèle d'extraction final.

Nous allons entrainer un Bi-LSTM qui fait parti de l'état de l'art des réseaux d'apprentissage profond.

***

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from snorkel.models import candidate_subclass
from snorkel.annotations import load_marginals
from snorkel.annotations import load_gold_labels
from snorkel import SnorkelSession

#from snorkel.learning.pytorch import LSTM
#from snorkel.learning.pytorch.rnn import LSTM

session = SnorkelSession()

# I. Chargement des données

On initialise notre schéma.

In [2]:
Partner = candidate_subclass('Partner', ['company1', 'company2'])

Chargement des labels d'entrainement probabilistes (_marginals_)

In [19]:
train_marginals = load_marginals(session, split=0)

Chargement des candidats

In [4]:
train_cands = session.query(Partner).filter(Partner.split == 0).all()
dev_cands   = session.query(Partner).filter(Partner.split == 1).all()
test_cands  = session.query(Partner).filter(Partner.split == 2).all()

Chargement des gold labels pour l'évaluation de notre modélisation.

In [5]:
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

# II. Entrainement d'un modèle LSTM d'extraction

In [26]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':              0.01,
    'embedding_dim':   100,
    'hidden_dim':      100,
    'n_epochs':        12,
    'dropout':         0.5,
    'rebalance':       0.25,
    'print_freq':      1,
    'seed':            1701
}

lstm = LSTM(n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=1369  #epochs=12  batch size=64
[LSTM] Epoch 1 (8.47s)	Average loss=0.691718	Dev F1=42.86
[LSTM] Epoch 2 (17.84s)	Average loss=0.666409	Dev F1=45.93
[LSTM] Epoch 3 (28.51s)	Average loss=0.647650	Dev F1=41.54
[LSTM] Epoch 4 (38.71s)	Average loss=0.647364	Dev F1=48.48
[LSTM] Epoch 5 (49.03s)	Average loss=0.637925	Dev F1=43.54
[LSTM] Epoch 6 (63.10s)	Average loss=0.637171	Dev F1=44.44
[LSTM] Epoch 7 (74.11s)	Average loss=0.634260	Dev F1=45.87
[LSTM] Epoch 8 (85.98s)	Average loss=0.626938	Dev F1=43.94
[LSTM] Epoch 9 (98.08s)	Average loss=0.621916	Dev F1=43.18
[LSTM] Epoch 10 (110.07s)	Average loss=0.623558	Dev F1=45.00
[LSTM] Epoch 11 (121.34s)	Average loss=0.618002	Dev F1=48.65
[LSTM] Model saved as <LSTM>
[LSTM] Epoch 12 (130.95s)	Average loss=0.616145	Dev F1=43.08
[LSTM] Training done (131.21s)
[LSTM] Loaded model <LSTM>


## Score sur jeu de développement

In [27]:
p, r, f1 = lstm.score(dev_cands, L_gold_dev)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.397, Recall: 0.628, F1 Score: 0.486


## Score sur Test

In [28]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.386, Recall: 0.835, F1 Score: 0.528
